# DYNAMIC ROUTING SIMULATOR

This takes you step by step for creating a dynamic routing simulator which leverages the Carto Overture Maps, Open Route Service as well as many of the advanced analytical functions within Snowflake.  Let's get started.

## Initial Setup

### Open Route Service Native
- Before you start, you will need to install the Route Optimisation Native app - which is available on the **Internal Market Place**.  Follow the instructions for the **Native App** installiation and then return here to try the functions out.


#### 1.  Simple Directions - allows simple directions from one point to another.
You can load multiple start/end points to get directions for multiple places). You also need to specify the method such as driving-car.  This function is leveraging a GET request from the api.  You will note that the function is written in python - the service itself has handy python snippets to help curate the function.  
click [here](https://openrouteservice.org/dev/#/api-docs/v2/directions/{profile}/post) to navigate to the directions service playground

I have decided to use cortex complete's **Anthropic LLM** -  **Claude** to provide an array of values which contain one nice hotel and one nice restaurant in **San Francisco**.

In [ ]:
select parse_json(SNOWFLAKE.CORTEX.COMPLETE('claude-3-5-sonnet','GIVE ME AN ARRAY THAT CONTAINS 1 hotel near union square and 1 restaurant near the sunset district IN SAN FRANCISCO.  only return the array.  Each point should be in this format: {"restaurants":["nice hotel","great restaurant"],"coordinates":[[longitude],[latitude]],[[longitude],[latitude]]] ')) points

In [ ]:
select points:restaurants start_destination,OPEN_ROUTE_SERVICE_V2.CORE.DIRECTIONS('driving-car',object_construct('coordinates',POINTS:coordinates)) GEO FROM {{random_points}}

In [ ]:
select * exclude GEO, GEO:features[0]:geometry::variant GEO, 
GEO:features[0]:properties:segments INSTRUCTIONS, 
DIV0(GEO:features[0]:properties:summary:distance::FLOAT,1000) DISTANCE_KM,
DIV0(GEO:features[0]:properties:summary:duration::FLOAT,60) DURATION_MINS

from

(select OPEN_ROUTE_SERVICE_V2.CORE.DIRECTIONS('driving-car',object_construct('coordinates',POINTS:coordinates)) GEO FROM {{random_points}})




I will now introduce you to the **pydeck** library.  This library allows you to visualise multiple map layers which can include polygons,linestrings and points.  Below is rendering a simple line string which gives me the path to drive to the Snowflake Office from my house

In [ ]:
import streamlit as st
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import *
from snowflake.snowpark.types import *
import pandas as pd
import pydeck as pdk
import json
session = get_active_session()


data = directions_try_out.to_df().select('GEO').to_pandas()
data["coordinates"] = data["GEO"].apply(lambda row: json.loads(row)["coordinates"])

vehicle_1_path = pdk.Layer(
type="PathLayer",
data=data,
pickable=True,
get_color=[253, 180, 107],
width_scale=20,
width_min_pixels=4,
width_max_pixels=7,
get_path="coordinates",
get_width=5)

view_state = pdk.ViewState(latitude=37.8199, longitude=-122.4783, zoom=12)

with st.container(height=700):
    st.pydeck_chart(pdk.Deck(layers=[vehicle_1_path],map_style=None,initial_view_state=view_state),height=700)

#### 2. Advanced directions - create routes which include multiple way points. 
Although the function above is nice and simple for directions that stops at only one place, vehicle route plans that contain multiple drop offs will need to effectively 'detour' between the starting point and destination.  This is where we utilise **way points**.

Each stop is loaded into one array.  You also need to specify the method such as driving-car.  This type of api uses a PUT request - you will see how this is articulated in the code below.

Lets now take you through calling the service which pushes in multiple way points.  Firstly, you will create a series of waypoints in an array.  Python is a great way to inject array variables into our sql function via a notebook for testing purposes.

Lets now take you through calling the service which pushes in multiple way points.  Firstly, you will create a series of waypoints in an array.  Python is a great way to inject array variables into our sql function via a notebook for testing purposes.

In [ ]:
select points:coordinates points, points:restaurants restaurants from (

select parse_json(SNOWFLAKE.CORTEX.COMPLETE('claude-3-5-sonnet','GIVE ME AN ARRAY THAT CONTAINS 10 restaurants IN SAN FRANCISCO.  only return the array.  Each point should be in this format: {"restaurants":["nice restaurant","great restaurant"],"coordinates":[[longitude],[latitude]],[[longitude],[latitude]]] ')) points)

In [ ]:
SELECT OPEN_ROUTE_SERVICE_V2.CORE.DIRECTIONS('driving-car',OBJECT_CONSTRUCT('coordinates',
    POINTS
))directions from {{five_random}}

In [ ]:
way_points = five_random.to_df()
way_points

We will then use Snowpark to persist the waypoints into a dataframe.  This will be used to render the points on a map. 

In [ ]:
import streamlit as st
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import *
from snowflake.snowpark.types import *
import pandas as pd
import pydeck as pdk
import json
session = get_active_session()


waypoints = way_points.join_table_function('flatten','POINTS').select(col('VALUE').alias('POINTS'))
restaurant_names = way_points.join_table_function('flatten','RESTAURANTS').select(col('VALUE').astype(StringType()).alias('RESTAURANT'))
waypoints = waypoints.select(col('POINTS')[0].astype(FloatType()).alias('LON'),
                             col('POINTS')[1].astype(FloatType()).alias('LAT'))

st.write(waypoints)

st.write(restaurant_names)

Now lets call the waypoint function which will return a line string containing the directions.  We do have the capability to extract the instructions for each waypoint - all this is stored in the same way as the simple directions service.  However, for this exercise we are interested in the line string only.  This will in turn form the second layer of the map.

In [ ]:
SELECT OPEN_ROUTE_SERVICE_V2.CORE.DIRECTIONS('driving-car',OBJECT_CONSTRUCT('coordinates',
    POINTS
)):features[0]:geometry GEO from {{five_random}}

We will now visualise the the directions with way points again using pydeck.  This incorporates two layers - one for the line string and the second for each way point 

In [ ]:
import streamlit as st
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import *
from snowflake.snowpark.types import *
import pandas as pd
import pydeck as pdk
import json
session = get_active_session()

restaurant_namespd = restaurant_names.to_pandas()
drop_offs = waypoints.to_pandas()
center = waypoints.agg(avg('LAT').alias('LAT'),avg('LON').alias('LON')).to_pandas()
drop_offs = pd.concat([drop_offs,restaurant_namespd],axis=1)
data = try_way_points.to_df().select('GEO').to_pandas()
data["coordinates"] = data["GEO"].apply(lambda row: json.loads(row)["coordinates"])
#st.write(drop_offs)
scatter = pdk.Layer(
            'ScatterplotLayer',
            data=drop_offs,
            opacity=0.8,
            get_position='[LON, LAT]',
            get_fill_color=[0,0,0],
            radius_scale=6,
            radius_min_pixels=5,
            radius_max_pixels=100,
            pickable=True)


vehicle_1_path = pdk.Layer(
type="PathLayer",
data=data,
pickable=False,
get_color=[253, 180, 107],
width_scale=20,
width_min_pixels=4,
width_max_pixels=7,
get_path="coordinates",
get_width=6)

tooltip = {
   "html": """<b>Name:</b> {RESTAURANT}""",
   "style": {
       "width":"50%",
        "backgroundColor": "steelblue",
        "color": "white",
       "text-wrap": "balance"
   }
}

view_state = pdk.ViewState(latitude=center.LAT.iloc[0], longitude=center.LON.iloc[0], zoom=12)
st.pydeck_chart(pdk.Deck(layers=[vehicle_1_path,scatter],tooltip=tooltip,map_style=None,initial_view_state=view_state,height=900))

#### 3. Route Optimisation Function - used to find the best optimal workload for each vehicle based on all the job requirements and the vehicles that are available.  

The job detailed requirements such as skills, capacity requirements and delivery slots are sent in the first array. Any available vehicle specifications are sent in the second array.

Now let's test this function out.  Below are two variables created for jobs and vehicles.  The optimisation service has documentation regarding the sorts of information you can pass through for both jobs and vehicles.  I am passing information such as a skill, the drop off location, capacity requirements and a skill.  The **skill** number needs to match to assign a job to a vehicle.  If the skill number on the job does not match any of the vehicles, the job will be flagged as **unassigned**.  This number could effectively relate to special requirement such as the vehicle must contain a fridge.

In [ ]:
jobs = [
  {
    "capacity": [
      2
    ],
    "id": 1,
    "location": [
      -4.613748000000000e-01,
      5.152569760000000e+01
    ],
    "skills": [
      1
    ],
    "time_window": [
      32400,
      36000
    ]
  },
  {
    "capacity": [
      2
    ],
    "id": 6,
    "location": [
      -3.355460000000000e-01,
      5.150566550000000e+01
    ],
    "skills": [
      1
    ],
    "time_window": [
      36000,
      54000
    ]
  }

]

vehicles = [
  {
    "capacity": [
      2
    ],
    "end": [
      -4.467074000000000e-01,
      5.146048020000000e+01
    ],
    "id": 1,
    "profile": "driving-hgv",
    "skills": [
      1
    ],
    "start": [
      -4.467074000000000e-01,
      5.146048020000000e+01
    ],
    "time_windows": [
      28800,
      61200
    ]
  },
]

In [ ]:
select parse_json(SNOWFLAKE.CORTEX.COMPLETE('claude-3-5-sonnet', concat('create 1 object containing 10 delivery jobs to restaurants in sanfrancisco area based on the following template',
{{jobs}}::text,'also create 1 object containing 1 random vehicle using this format',{{vehicles}}::text,'return only the json results'))) JOBS

**Now** we will test the function out - we will apply the two variant variables to the function and a json object will be returned.

In [ ]:
select OPEN_ROUTE_SERVICE_V2.CORE.OPTIMIZATION(
    JOBS:jobs, 
   JOBS:vehicles) 
 OPTIMIZATION from {{fake_optimisation}}

Let's now have a look at the json object. You will see that the array object is broken down into routes.  Each route will be assigned a vehicle and that vehicle will be allocated delivery locations and times.  There will also be infomation about the duration and may include details such as waiting times.

In [ ]:
with st.container(height=500):
    st.code(test_route_optimisation.to_df().collect()[0][0])

We will use standard Geospatial functions to collect all the points into one row.  This will be used to create a line in pydeck.  In addition, I have used ST_ENVELOPE to create a bounding box for the collection of points.  This is so I can work out the overall centre point of each delivery location.  This will be useful for centering the rendered pydeck map.

In [ ]:
from snowflake.snowpark.functions import *
routes = test_route_optimisation.to_df().select(col('OPTIMIZATION')['routes'][0]['steps'].alias('steps'))

routes = routes.join_table_function('flatten',col('steps')).select('VALUE')
routes = routes.select(col('VALUE')['arrival'].alias('ARRIVAL'),
              col('VALUE')['arrival'].alias('duration'),
              col('VALUE')['load'].alias('load'),
              col('VALUE')['id'].alias('Job ID'),
              col('VALUE')['location'].alias('Location'),
              col('VALUE')['service'].alias('Service'))
            
line = routes.select(call_function('ST_COLLECT',call_function('st_makepoint',col('Location')[0],col('Location')[1])).alias('LINE')) 
line = line.with_column('CENTROID',call_function('ST_CENTROID',(call_function('ST_ENVELOPE',col('LINE')))))
line

We will view the journey by combining all drop offs.  Of course the journey will involve roads, that is where the waypoint function will be used - the returned results of each drop off will be pushed into the waypoint function.  The pydeck visualisation compares straight line with directions as two distinct layers

In [ ]:
import streamlit as st
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import *
from snowflake.snowpark.types import *
import pandas as pd
import pydeck as pdk
import json
session = get_active_session()

center = line.select(call_function('ST_X',col('CENTROID')).alias('X'),
                     call_function('ST_Y',col('CENTROID')).alias('Y')).limit(1).to_pandas()

LAT = center.Y.iloc[0]
LON = center.X.iloc[0]

directions = line.select(call_function('st_asgeojson',col('LINE'))['coordinates'].alias('LINE'))
st.write(directions)
directions = directions.with_column('line',
                                    call_function('OPEN_ROUTE_SERVICE_V2.CORE.DIRECTIONS',
                                                  'driving-car',
                                                      object_construct(lit('coordinates'),col('line')))['features'][0]['geometry'])

st.write(directions)
data = directions.select('LINE').to_pandas()
data["coordinates"] = data["LINE"].apply(lambda row: json.loads(row)["coordinates"])
data2 = line.to_pandas()
data2["coordinates"] = data2["LINE"].apply(lambda row: json.loads(row)["coordinates"])

vehicle_1_path = pdk.Layer(
type="PathLayer",
data=data,
pickable=True,
get_color=[253, 180, 107],
width_scale=20,
width_min_pixels=4,
width_max_pixels=7,
get_path="coordinates",
get_width=5)

vehicle_1_straight_line = pdk.Layer(
type="PathLayer",
data=data2,
pickable=True,
get_color=[255,0, 107],
width_scale=20,
width_min_pixels=4,
width_max_pixels=7,
get_path="coordinates",
get_width=5)

view_state = pdk.ViewState(latitude=LAT, longitude=LON, zoom=10)
st.markdown('Waypoints joined together as a straight line')
st.pydeck_chart(pdk.Deck(layers=[vehicle_1_path,vehicle_1_straight_line],map_style=None,initial_view_state=view_state,height=900))

#### 4. Isochrone Function.  
This is to create a polygon which creates a catchment boundary based on how long it would take to get there.  Orders may only be accepted if you can fullfill them based on how quickly drivers can get anywhere within a given timeframe.  Isochrones could be utilised for other usecases outside of route optimisation such as market analysis in a given catchment.

The function is then tested by focussing on a specific point.  It will generate a polygon which represents everywhere a driver can get to within a 20 minute timeframe.

In [ ]:
select OPEN_ROUTE_SERVICE_V2.CORE.ISOCHRONES('driving-car', -122.4783,37.8199, 20) boo, boo:features[0]:geometry GEO, ST_CENTROID(TO_GEOGRAPHY(GEO)) CENTROID, ST_X(CENTROID) LON, ST_Y(CENTROID) LAT

This is an example of what an isochrone looks like in pydeck.

In [ ]:
import streamlit as st
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import *
from snowflake.snowpark.types import *
import pandas as pd
import pydeck as pdk
import json
session = get_active_session()


data = isochrones_try.to_df().select('GEO','LON','LAT').to_pandas()
LAT = data.LAT.iloc[0]
LON = data.LON.iloc[0]
data["coordinates"] = data["GEO"].apply(lambda row: json.loads(row)["coordinates"])


isochrone = pdk.Layer(
"PolygonLayer",
data,
opacity=0.7,
get_polygon="coordinates",
filled=True,
get_line_color=[41,181,232],
get_fill_color=[200,230,242],
get_line_width=10,
line_width_min_pixels=6,
auto_highlight=True,
pickable=False)

view_state = pdk.ViewState(latitude=LAT, longitude=LON, zoom=10)
st.pydeck_chart(pdk.Deck(layers=[isochrone],map_style=None,initial_view_state=view_state,height=900))

Let's now use cortex to find 15 restaurants inside the generated polygon

In [ ]:
select *,SNOWFLAKE.CORTEX.COMPLETE('claude-3-5-sonnet', concat('provide 10 restaurants inside the following polygon.',GEO::TEXT, 'return the Longitude, Latitude and Restaurant name as json only')) RESTAURANTS FROM {{isochrones_try}}

In [ ]:
SELECT VALUE:name::TEXT RESTAURANT, VALUE:longitude::FLOAT LON, VALUE:latitude::FLOAT LAT FROM {{restaurants_in_isochrone}}, LATERAL FLATTEN (parse_json(RESTAURANTS):restaurants)

In [ ]:
import streamlit as st
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import *
from snowflake.snowpark.types import *
import pandas as pd
import pydeck as pdk
import json
session = get_active_session()


data = isochrones_try.to_df().select('GEO','LON','LAT').to_pandas()
restaurants = found_restaurants.to_pandas()

scatter = pdk.Layer(
            'ScatterplotLayer',
            data=restaurants,
            opacity=0.8,
            get_position='[LON, LAT]',
            get_fill_color=[0,0,0],
            radius_scale=6,
            radius_min_pixels=5,
            radius_max_pixels=100,
            pickable=True)

LAT = data.LAT.iloc[0]
LON = data.LON.iloc[0]
data["coordinates"] = data["GEO"].apply(lambda row: json.loads(row)["coordinates"])

isochrone = pdk.Layer(
"PolygonLayer",
data,
opacity=0.7,
get_polygon="coordinates",
filled=True,
get_line_color=[41,181,232],
get_fill_color=[200,230,242],
get_line_width=10,
line_width_min_pixels=6,
auto_highlight=True,
pickable=False)

view_state = pdk.ViewState(latitude=LAT, longitude=LON, zoom=10)
st.pydeck_chart(pdk.Deck(layers=[isochrone,scatter],map_style=None,initial_view_state=view_state,height=900))

## Create Base Data Table for App
Now we will focus on putting components together for a streamlit application.  We are aiming to create a dynamic routing analyser.  For this, we will need to generate jobs, vehicles and depots based on any location in the world.  Carto overture maps will help as it contains points of interests anywhere in the world. Cortex complete is also used for a simple 'geocoding' mechanism where a user will type in a key word and cortex will generate the coordinates that matches what central location the user wants to base the routing against

#### 1. Job Template
This template will be used to simulate jobs.  You can change the base values in this notebook - which will effect what sample jobs will be generated.  This template generates a sample of 29 jobs.  The location of these jobs and industry will be dynamic and depends on user interaction with the app.

In [ ]:
CREATE OR REPLACE TABLE DEFAULT_SCHEMA.JOB_TEMPLATE (

ID INT AUTOINCREMENT PRIMARY KEY,
slot_start INT NOT NULL,
slot_end INT,
skills INT,
product STRING,
status STRING DEFAULT 'active'

)



In [ ]:
INSERT INTO DEFAULT_SCHEMA.JOB_TEMPLATE (slot_start, slot_end, skills, product, status) VALUES
(9, 10, 1, 'pa', 'active'),
(11, 15, 2, 'pb', 'active'),
(16, 18, 2, 'pb', 'active'),
(11, 13, 3, 'pc', 'active'),
(7, 16, 3, 'pc', 'active'),
(10, 15, 2, 'pa', 'active'),
(10, 15, 2, 'pa', 'active'),
(7, 16, 1, 'pa', 'active'),
(9, 18, 2, 'pb', 'active'),
(13, 18, 2, 'pb', 'active'),
(13, 18, 2, 'pb', 'active'),
(13, 18, 1, 'pa', 'active'),
(13, 18, 1, 'pa', 'active'),
(13, 18, 1, 'pa', 'active'),
(13, 18, 3, 'pc', 'active'),
(11, 15, 2, 'pb', 'active'),
(16, 18, 2, 'pb', 'active'),
(11, 13, 1, 'pa', 'active'),
(7, 16, 1, 'pa', 'active'),
(10, 15, 2, 'pb', 'active'),
(10, 15, 2, 'pb', 'active'),
(7, 16, 1, 'pa', 'active'),
(9, 18, 2, 'pb', 'active'),
(13, 18, 2, 'pb', 'active'),
(13, 18, 2, 'pb', 'active'),
(13, 18, 1, 'pa', 'active'),
(13, 18, 1, 'pa', 'active'),
(13, 18, 1, 'pa', 'active'),
(13, 18, 3, 'pc', 'active');


In [ ]:
SELECT * FROM DEFAULT_SCHEMA.JOB_TEMPLATE

#### 2. Create a base table with search optimisation to filter quickly on categories and geographies

Here we are using the places dataset which is provided by carto overture maps.  We are copying the data to add search optimisation on the categories of place as well as the geography.  We will be leveraging the 'Search' function in the app to search through columns that may be relevant to the selected industry.

In [ ]:
CREATE OR REPLACE TABLE DEFAULT_SCHEMA.places as 

select 

any_value(GEOMETRY) GEOMETRY,

PHONES:list[0]['element']::text PHONES,
CATEGORIES:primary::text CATEGORY,
NAMES:primary::text NAME,
ADDRESSES:list[0]['element'] ADDRESS,

array_agg(value:element) as ALTERNATE 

from

(SELECT PHONES,CATEGORIES,NAMES,ADDRESSES,GEOMETRY,

categories:alternate:list AS LIST

from DEFAULT_DATABASE.DEFAULT_SCHEMA.POI_IN_CALIFORNIA), 

--where CATEGORIES:primary is not null),
LATERAL FLATTEN(LIST) GROUP BY ALL;

ALTER TABLE DEFAULT_SCHEMA.places ADD SEARCH OPTIMIZATION ON EQUALITY(ALTERNATE);
ALTER TABLE DEFAULT_SCHEMA.places ADD SEARCH OPTIMIZATION ON GEO(GEOMETRY);

SELECT * FROM DEFAULT_SCHEMA.PLACES limit 3

In [ ]:
select * EXCLUDE (GEOMETRY,GEO),ST_X(GEOMETRY) LON,ST_Y(GEOMETRY) LAT from DEFAULT_SCHEMA.PLACES A INNER JOIN
(select GEO from {{isochrones_try}}) B ON

ST_WITHIN(TO_GEOGRAPHY(A.GEOMETRY),TO_GEOGRAPHY(B.GEO))

where CATEGORY = 'hotel'

Here, we are using the Scatter Plot layer to only view places by the isochrone 

In [ ]:
center = isochrones_try.to_df().limit(1).to_pandas()

BLAT = center.LAT.iloc[0]
BLON = center.LON.iloc[0]

context = pdk.Layer(
'ScatterplotLayer',
places_on_isochrone.to_pandas(),
get_position=['LON', 'LAT'],
filled=True,
stroked=False,
radius_min_pixels=6,
radius_max_pixels=20,
auto_highlight=True,
get_fill_color=[41, 181, 232],
pickable=True)
st.divider()

view_state = pdk.ViewState(latitude=BLAT, longitude=BLON, zoom=10)
st.pydeck_chart(pdk.Deck(layers=[context],map_style=None,initial_view_state=view_state,height=900))

#### Below are all the categories you could use to simulate distribution points
The app will search through names, categories and alternative categories in order to retrieve the relevant data

In [ ]:
SELECT DISTINCT CATEGORY FROM (
select DISTINCT VALUE::TEXT CATEGORY FROM DEFAULT_SCHEMA.PLACES, LATERAL FLATTEN (ALTERNATE)

UNION 

SELECT DISTINCT CATEGORY FROM DEFAULT_SCHEMA.PLACES) WHERE SEARCH((CATEGORY),'food')

### 4. Creating an Industry Lookup

Depending on the selected industry, the app will 'lookup' specifics in order to retrieve/ filter the right information.  It is also used to generate relevant product categories. In this notebook, more industries can be added.

In [ ]:
CREATE OR REPLACE TABLE DEFAULT_SCHEMA.LOOKUP (
    INDUSTRY STRING,
    PA STRING,
    PB STRING,
    PC STRING,
    IND ARRAY,
    IND2 ARRAY,
    CTYPE ARRAY,
    STYPE ARRAY
);

INSERT INTO DEFAULT_SCHEMA.LOOKUP (INDUSTRY, PA, PB, PC, IND,IND2, CTYPE, STYPE) 
SELECT
    'healthcare', 
    'flammable', 
    'sharps', 
    'temperature-controlled', 
    ARRAY_CONSTRUCT('hospital health pharmaceutical drug healthcare pharmacy surgical'), 
    ARRAY_CONSTRUCT('supplies warehouse depot distribution wholesaler distributors'), 
    ARRAY_CONSTRUCT('hospital', 'family_practice', 'dentist','pharmacy'), 
    ARRAY_CONSTRUCT('Can handle potentially explosive goods', 'Can handle instruments that could be used as weapons', 'Has a fridge')
UNION ALL
SELECT
    'Food', 
    'Fresh Food Order', 
    'Frozen Food Order', 
    'Non Perishable Food Order', 
    ARRAY_CONSTRUCT('food vegatables meat vegatable'),
    ARRAY_CONSTRUCT('wholesaler warehouse factory processing distribution distributors'), 
    ARRAY_CONSTRUCT('supermarket', 'restaurant', 'butcher_shop'), 
    ARRAY_CONSTRUCT('Can deliver Fresh Food', 'Has a Fridge', 'Premium Delivery')
UNION ALL
SELECT
    'Cosmetics', 
    'Hair Products', 
    'Electronic Goods', 
    'Make-up', 
    ARRAY_CONSTRUCT('hair cosmetics make-up beauty'),
    ARRAY_CONSTRUCT('wholesaler warehouse factory supplies distribution distributors'), 
    ARRAY_CONSTRUCT('supermarket', 'outlet', 'fashion'), 
    ARRAY_CONSTRUCT('Can deliver Fresh Food', 'Has a Fridge', 'Premium Delivery')

    ;



In [ ]:
SELECT * FROM DEFAULT_SCHEMA.LOOKUP

#### Here is an example of how the lookup workes for healthcare for flammable products

In [ ]:
select * from DEFAULT_SCHEMA.JOB_TEMPLATE a 

inner join

(select PA from lookup where industry = 'healthcare') b

on PRODUCT = 'pa'

## Streamlit App

Now you have setup the functions and standing data tables, you can now have a go at running the streamlit app.  The streamlit application is an example of what a basic route optimisation simulator could look like.  Feel free to make any changes as you see fit.

- Within Projects / Streamlit you should see a new application
- Click on the app to run the route optimisation simulator